# Multi-Beam Payload: Capacity Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jman4162/opensatcom/blob/main/notebooks/03_multibeam_payload.ipynb)

Define a 4-beam satellite payload, compute SINR maps, and visualize with interactive heatmaps.

In [ ]:
# Install opensatcom (uncomment for Colab)
# !pip install -q opensatcom

In [ ]:
import numpy as np
from opensatcom.core.models import (
    RFChainModel, Scenario, Terminal, PropagationConditions
)
from opensatcom.antenna.cosine import CosineRolloffAntenna
from opensatcom.antenna.parametric import ParametricAntenna
from opensatcom.propagation import FreeSpacePropagation
from opensatcom.payload.beam import Beam
from opensatcom.payload.beamset import BeamSet
from opensatcom.payload.capacity import compute_beam_map

## 1. Define Beams

Four beams with cosine rolloff antennas, offset in azimuth/elevation.

In [ ]:
beam_configs = [
    {"beam_id": "B1", "az": -2.0, "el": 32.0, "color": "red"},
    {"beam_id": "B2", "az": 2.0, "el": 32.0, "color": "blue"},
    {"beam_id": "B3", "az": -2.0, "el": 28.0, "color": "green"},
    {"beam_id": "B4", "az": 2.0, "el": 28.0, "color": "orange"},
]

beams = []
for bc in beam_configs:
    antenna = CosineRolloffAntenna(
        peak_gain_dbi=34.0,
        theta_3db_deg=3.0,
        sidelobe_floor_dbi=-20.0,
        boresight_az_deg=bc["az"],
        boresight_el_deg=bc["el"],
    )
    beams.append(Beam(
        beam_id=bc["beam_id"],
        az_deg=bc["az"],
        el_deg=bc["el"],
        tx_power_w=50.0,
        antenna=antenna,
    ))

print(f"Defined {len(beams)} beams")
for b in beams:
    print(f"  {b.beam_id}: az={b.az_deg}, el={b.el_deg}")

## 2. Create BeamSet and Compute Beam Map

In [ ]:
scenario = Scenario(
    name="Ka-band DL", direction="downlink",
    freq_hz=20e9, bandwidth_hz=500e6,
    polarization="RHCP", required_metric="ebn0_db", required_value=5.0,
)

beamset = BeamSet(
    beams, scenario,
    FreeSpacePropagation(),
    RFChainModel(tx_power_w=50.0, tx_losses_db=1.0, rx_noise_temp_k=150.0),
)

grid_az = np.arange(-5.0, 6.0, 0.5)
grid_el = np.arange(25.0, 36.0, 0.5)

rx_antenna = ParametricAntenna(gain_dbi=34.0)
rx_terminal = Terminal("Ground", 38.9, -77.0, 0.0, system_noise_temp_k=290.0)

from opensatcom.geometry.slant import slant_range_m
range_m = slant_range_m(0.0, 35_786_000.0, 30.0)

beam_map = compute_beam_map(
    beamset, grid_az, grid_el,
    rx_antenna, rx_terminal, range_m,
    PropagationConditions(),
)

print(f"Grid points:   {len(beam_map)}")
print(f"Mean SINR:     {beam_map.sinr_db_mean:.2f} dB")
print(f"Mean margin:   {beam_map.margin_db_mean:.2f} dB")

## 3. Interactive Beam Map Visualization

In [ ]:
from opensatcom.viz.heatmaps import plot_beam_map_interactive

df = beam_map.to_dataframe()
fig = plot_beam_map_interactive(df, metric="sinr_db", title="SINR Coverage Map")
fig.show()

In [ ]:
fig = plot_beam_map_interactive(df, metric="margin_db", title="Margin Coverage Map")
fig.show()

---

**Next:** See `04_propagation_models.ipynb` for propagation model comparison.